In [21]:
import plotly.graph_objects as go
import numpy as np

file_path = "./BENCHMARK2"
paths = open(file_path, "r").read().split("\n")
paths_dic = {paths[2*i+1].replace(' ', ''):paths[2*i+2].split(',')[:-1] for i in range(int(len(paths)/2))}
paths_dic = {path_name:list(map((lambda x : float(x)), paths_dic[path_name])) for path_name in paths_dic.keys()}
T = 1
N = 1000
dt = {path_name:T/(len(paths_dic[path_name])-1) for path_name in paths_dic}
grid = np.linspace(0, T, N+1)[:-1]
possible_paths = paths_dic.keys()
def f_map(path, T):
    return (lambda t : path[int(len(path)*t/T)])

In [22]:
possible_paths

dict_keys(['BROWNIAN', 'IGBM_Parabola', 'IGBM_Log_ODE', 'log_integration_IGBM', 'polyparabola_integration_IGBM', 'poly1_integration_IGBM', 'poly2_integration_IGBM', 'poly3_integration_IGBM', 'poly2_integration_CASE1', 'log_integration_CASE1', 'poly2_integration_CASE2', 'log_integration_CASE2', 'poly5_integration_CASE2', 'poly2_integration_CASE3', 'log_integration_CASE3', 'poly2_integration_CASE4', 'log_integration_CASE4', 'poly2_integration_CASE5', 'log_integration_CASE5', 'log_integration_CASE6'])

In [23]:
"""
Brownian path
"""
fig = go.Figure()
for path_name in ['BROWNIAN']:
    path = paths_dic[path_name]
    fig.add_trace(go.Scatter(x=grid, y=np.array(list(map(f_map(path, 1), grid))),
                    mode='lines',
                    name=path_name))
fig.update_layout(title="W_t")
fig.show()

In [24]:
"""
Inhomogeneous Geometric Brownian Motion
dy_t = a(b-y_t) + sigma y_t dW_t
IGBM_Parabola: 
    Closed-form formula linking y_t to y_{t+dt}, replace W_t by Parabola_t
IGBM_Log_ODE: 
    Log ODE method
polyparabola_integration_IGBM:
    Same as the poly2_integration one but using dParabola_u = (W_1 + (6-u)H_1)du
polyn_integration_IGBM:
    Numerical scheme obtained by replacing dW_t by dW^n_t
    
    
Different SDEs
Case1: 
dy_t = y_t^2 dW_t, y_0 = 1
=> y_t = 1/(1-W_t)

Case2 (GBM):
dy_t = y_tdt + y_t dW_t, y_0 = 2
=> y_t = 2exp(1/2t + W_t)


Case3:
dy_t = sin(y_t) dW_t, y_0 = PI/2

Case4 (CIR Process):
dy_t = (1.2-y_t) + 0.2sqrt(y_t)dW_t, y_0 = 1.

Case5:
A modified Wright-Fishcer Model
dy_t = y_t(1-y_t)dt + sqrt(y_t(1-y_t))dW_t, y_0 € ]0,1[, y_0 = 0.3

Case6:
dy_t = dt + 2sqrt(y_t)dW_t
y_t = (sqrt(y_0) + W_t)^2
"""
fig = go.Figure()
for path_name in possible_paths:
    if path_name in possible_paths:
        path = paths_dic[path_name]
        fig.add_trace(go.Scatter(x=grid, y=np.array(list(map(f_map(path, 1), grid))),
                        mode='lines',
                        name=path_name))
#Case1
f = (lambda w : 1/(1-w))
fig.add_trace(go.Scatter(x=grid, y=f(np.array(list(map(f_map(paths_dic['BROWNIAN'], 1), grid)))),
                        mode='lines',
                        name='Case1:y_0/(1-Wt)'))    
#Case2
f = (lambda t, w : 2*np.exp(0.5*t + w))
fig.add_trace(go.Scatter(x=grid, y = f(grid, np.array(list(map(f_map(paths_dic['BROWNIAN'], 1), grid)))),
                        mode='lines',
                        name='Case2:y_0exp(1/2t + Wt)'))

#Case6
f = (lambda w : (1.2**0.5 + w)**2)
fig.add_trace(go.Scatter(x=grid, y = f(np.array(list(map(f_map(paths_dic['BROWNIAN'], 1), grid)))),
                        mode='lines',
                        name='Case6:(sqrt(y_0) + Wt)^2'))
fig.update_layout(title="Closed forms, Numerical simulations")

fig.show()